# Democracy chart

#### Import libraries

In [17]:
# datascience
import numpy as np 
import pandas as pd 
import bisect

# plot
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib.image as image
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import flag
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

# files
import urllib.request
import cairosvg

import seaborn as sns
from IPython.core.display import HTML

pd.set_option('display.max_colwidth', -1)

<ipython-input-17-9b9f666f8904>:29: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



## Datas 

#### Covid datas

In [18]:
covid_data = pd.read_csv("../data/covid19.csv")
covid_data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,NaN,0.0,0.0,NaN,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,NaN,0.0,0.0,NaN,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,NaN,0.0,0.0,NaN,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,NaN,0.0,0.0,NaN,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,NaN,0.0,0.0,NaN,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83


In [19]:
covid_df = covid_data[['iso_code','date', 'location', 'continent', 'total_deaths', 'total_cases', 'population']]

In [20]:
covid_df = covid_df[covid_df.location != 'World']
covid_df = covid_df[covid_df.location != 'International']
covid_df.head()

,iso_code,date,location,continent,total_deaths,total_cases,population
0,AFG,2019-12-31,Afghanistan,Asia,0.0,0.0,38928341.0
1,AFG,2020-01-01,Afghanistan,Asia,0.0,0.0,38928341.0
2,AFG,2020-01-02,Afghanistan,Asia,0.0,0.0,38928341.0
3,AFG,2020-01-03,Afghanistan,Asia,0.0,0.0,38928341.0
4,AFG,2020-01-04,Afghanistan,Asia,0.0,0.0,38928341.0


In [21]:
covid_last_df = covid_df[covid_df.date == '2020-08-16']
covid_last_df.head()

,iso_code,date,location,continent,total_deaths,total_cases,population
229,AFG,2020-08-16,Afghanistan,Asia,1370.0,37551.0,38928341.0
392,ALB,2020-08-16,Albania,Europe,225.0,7260.0,2877800.0
624,DZA,2020-08-16,Algeria,Africa,1360.0,38133.0,43851043.0
788,AND,2020-08-16,Andorra,Europe,53.0,989.0,77265.0
938,AGO,2020-08-16,Angola,Africa,86.0,1852.0,32866268.0


In [22]:
covid_last_df = covid_last_df.drop(['date'], axis=1)
covid_last_df.head()

,iso_code,location,continent,total_deaths,total_cases,population
229,AFG,Afghanistan,Asia,1370.0,37551.0,38928341.0
392,ALB,Albania,Europe,225.0,7260.0,2877800.0
624,DZA,Algeria,Africa,1360.0,38133.0,43851043.0
788,AND,Andorra,Europe,53.0,989.0,77265.0
938,AGO,Angola,Africa,86.0,1852.0,32866268.0


In [44]:
us = covid_last_df[covid_last_df.location == 'Congo']
us

,iso_code,location,continent,total_deaths,total_cases,population
7961,COG,Congo,Africa,60.0,3745.0,5518092.0


In [40]:
covid_last_df.location.values

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus',
       'Czech Republic', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'Faeroe Islands', 'Falkland Isla

#### Flags

In [24]:
flags = pd.read_csv("../data/Country_Flags.csv")
flags.head()

,Country,Images File Name,ImageURL
0,Afghanistan,Flag_of_Afghanistan.svg,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,Albania,Flag_of_Albania.svg,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,Algeria,Flag_of_Algeria.svg,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,Andorra,Flag_of_Andorra.svg,https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg
4,Angola,Flag_of_Angola.svg,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg


In [25]:
# Rename Country to location to match the first dataframe
flags_df = flags.rename(columns ={'Country':'location'})
# Drop files name
flags_df = flags_df[['location', 'ImageURL']]

flags_df.head()

,location,ImageURL
0,Afghanistan,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,Albania,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,Algeria,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,Andorra,https://upload.wikimedia.org/wikipedia/commons/1/19/Flag_of_Andorra.svg
4,Angola,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg


#### Longitude and latitude

In [26]:
long_lat = pd.read_csv("../data/longitude.csv")
long_lat.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [27]:
long_lat_df = long_lat.rename(columns ={'country':'location'})
long_lat_df = long_lat_df[['latitude', 'longitude', 'location']]
long_lat_df.head()

,latitude,longitude,location
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


#### Democracy datas

In [28]:
democracy = pd.read_csv("../data/political_regime.csv")
democracy.head()

,Entity,Code,Year,Political Regime (OWID based on Polity IV and Wimmer & Min) (Score)
0,Afghanistan,AFG,1816,-6
1,Afghanistan,AFG,1817,-6
2,Afghanistan,AFG,1818,-6
3,Afghanistan,AFG,1819,-6
4,Afghanistan,AFG,1820,-6


In [29]:
democracy_df = democracy[democracy.Year==2015]
democracy_df = democracy_df.rename(
    columns =
    {
    'Entity':'location',
    'Political Regime (OWID based on Polity IV and Wimmer & Min) (Score)': 'democracy_score'})

democracy_df = democracy_df.drop(['Year', 'Code'], axis=1)
democracy_df.head()

,location,democracy_score
176,Afghanistan,-1
375,Albania,9
575,Algeria,2
775,Angola,-2
966,Argentina,9


#### Merging

In [98]:
covid_democracy = pd.merge(covid_last_df, democracy_df, on='location')
covid_democracy = pd.merge(covid_democracy, flags_df, on='location')
covid_democracy = pd.merge(covid_democracy, long_lat_df, on='location')
covid_democracy.head()

,iso_code,location,continent,total_deaths,total_cases,population,democracy_score,ImageURL,latitude,longitude
0,AFG,Afghanistan,Asia,1370.0,37551.0,38928341.0,-1,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,33.939110,67.709953
1,ALB,Albania,Europe,225.0,7260.0,2877800.0,9,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,41.153332,20.168331
2,DZA,Algeria,Africa,1360.0,38133.0,43851043.0,2,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg,28.033886,1.659626
3,AGO,Angola,Africa,86.0,1852.0,32866268.0,-2,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,-11.202692,17.873887
4,ARG,Argentina,South America,5565.0,282424.0,45195777.0,9,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,-38.416097,-63.616672


In [99]:
covid_democracy['score'] = [bisect.bisect_left([-6, -1,5,10], p)
                            for p in covid_democracy.democracy_score]

covid_democracy = covid_democracy.drop(['democracy_score'], axis=1)

covid_democracy

,iso_code,location,continent,total_deaths,total_cases,population,ImageURL,latitude,longitude,score
0,AFG,Afghanistan,Asia,1370.0,37551.0,38928341.0,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,33.939110,67.709953,1
1,ALB,Albania,Europe,225.0,7260.0,2877800.0,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,41.153332,20.168331,3
2,DZA,Algeria,Africa,1360.0,38133.0,43851043.0,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg,28.033886,1.659626,2
3,AGO,Angola,Africa,86.0,1852.0,32866268.0,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,-11.202692,17.873887,1
4,ARG,Argentina,South America,5565.0,282424.0,45195777.0,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,-38.416097,-63.616672,3
...,...,...,...,...,...,...,...,...,...,...
146,VEN,Venezuela,South America,276.0,32607.0,28435943.0,https://upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Venezuela.svg,6.423750,-66.589730,2
147,VNM,Vietnam,Asia,22.0,934.0,97338583.0,https://upload.wikimedia.org/wikipedia/commons/2/21/Flag_of_Vietnam.svg,14.058324,108.277199,0
148,YEM,Yemen,Asia,529.0,1862.0,29825968.0,https://upload.wikimedia.org/wikipedia/commons/8/89/Flag_of_Yemen.svg,15.552727,48.516388,2
149,ZMB,Zambia,Africa,260.0,9186.0,18383956.0,https://upload.wikimedia.org/wikipedia/commons/0/06/Flag_of_Zambia.svg,-13.133897,27.849332,3


In [238]:
covid_democracy['Regimes'] = covid_democracy['score'].replace([0, 1, 2, 3],
                                                            ['Autocracy', 'Closed Anocracy', 
                                                             'Open Anocracy','Democracy'])

In [64]:
covid_democracy.to_csv('../data/covid_democracy.csv', index=False)

In [101]:
covid_democracy.head()

,iso_code,location,continent,total_deaths,total_cases,population,ImageURL,latitude,longitude,score,Regimes
0,AFG,Afghanistan,Asia,1370.0,37551.0,38928341.0,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg,33.939110,67.709953,1,Closed Anocracy
1,ALB,Albania,Europe,225.0,7260.0,2877800.0,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg,41.153332,20.168331,3,Democracy
2,DZA,Algeria,Africa,1360.0,38133.0,43851043.0,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg,28.033886,1.659626,2,Open Anocracy
3,AGO,Angola,Africa,86.0,1852.0,32866268.0,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg,-11.202692,17.873887,1,Closed Anocracy
4,ARG,Argentina,South America,5565.0,282424.0,45195777.0,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg,-38.416097,-63.616672,3,Democracy


## Table 

In [295]:
covid_table = covid_democracy.drop(['iso_code','latitude','longitude'], axis=1)
covid_table = covid_table[['score','location', 'continent','total_deaths', 'total_cases', 'Regimes', 'ImageURL']]
covid_table.head()

,score,location,continent,total_deaths,total_cases,Regimes,ImageURL
0,1,Afghanistan,Asia,1370.0,37551.0,Closed Anocracy,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,3,Albania,Europe,225.0,7260.0,Democracy,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,2,Algeria,Africa,1360.0,38133.0,Open Anocracy,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,1,Angola,Africa,86.0,1852.0,Closed Anocracy,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg
4,3,Argentina,South America,5565.0,282424.0,Democracy,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg


In [308]:
covid_table = covid_table.rename(columns={
    'location':'Country',
    'continent': 'Continent',
    'total_deaths': 'Total deaths',
    'total_cases': 'Total cases',
    'Regimes': 'Regimes',
    'ImageURL': 'Flags',
})

In [241]:
def highlight(s):
    if s.score == 0:
        return ['background-color: rgb(227,6,19); color: #e6e6e6; font-weight: bold; letter-spacing: 1px; padding: 12px 8px; border-top: 1px solid rgba(255,255,255,0.5); border-left: 1px solid rgba(255,255,255,0.5);']*7
    elif s.score == 1:
        return ['background-color: rgb(227, 126, 2);font-weight: bold; letter-spacing: 1px; padding: 12px 8px; border-top: 1px solid rgba(255,255,255,0.5); border-left: 1px solid rgba(255,255,255,0.5);']*7
    elif s.score == 2:
        return ['background-color: rgb(58, 207, 107);font-weight: bold; letter-spacing: 1px; padding: 12px 8px; border-top: 1px solid rgba(255,255,255,0.5); border-left: 1px solid rgba(255,255,255,0.5);']*7
    else:
        return ['background-color: rgb(39, 141, 204);font-weight: bold; letter-spacing: 1px; padding: 12px 8px; border-top: 1px solid rgba(255,255,255,0.5); border-left: 1px solid rgba(255,255,255,0.5);']*7

In [247]:
covid_table.style\
    .apply(highlight, axis=1)\
    .set_table_styles([
        {
            'selector': 'th',
            'props': [
                ('color', '#fff'),
                ('font-weight', 'bold'),
                ('background', '#313131'),
                ('border', 'none'),
                ('border-left', '1px solid rgba(255,255,255,0.5)')
            ]
        },
        {
            'selector': 'tr > th:first-child',
            'props': [
                ('display', 'none')
            ]
        }
    ])

,location,continent,total_deaths,total_cases,score,Regimes,ImageURL
0,Afghanistan,Asia,1370.000000,37551.000000,1,Closed Anocracy,https://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg
1,Albania,Europe,225.000000,7260.000000,3,Democracy,https://upload.wikimedia.org/wikipedia/commons/3/36/Flag_of_Albania.svg
2,Algeria,Africa,1360.000000,38133.000000,2,Open Anocracy,https://upload.wikimedia.org/wikipedia/commons/7/77/Flag_of_Algeria.svg
3,Angola,Africa,86.000000,1852.000000,1,Closed Anocracy,https://upload.wikimedia.org/wikipedia/commons/9/9d/Flag_of_Angola.svg
4,Argentina,South America,5565.000000,282424.000000,3,Democracy,https://upload.wikimedia.org/wikipedia/commons/1/1a/Flag_of_Argentina.svg
5,Armenia,Asia,817.000000,41495.000000,2,Open Anocracy,https://upload.wikimedia.org/wikipedia/commons/2/2f/Flag_of_Armenia.svg
6,Australia,Oceania,379.000000,23035.000000,3,Democracy,https://upload.wikimedia.org/wikipedia/commons/8/88/Flag_of_Australia_%28converted%29.svg
7,Austria,Europe,728.000000,23211.000000,3,Democracy,https://upload.wikimedia.org/wikipedia/commons/4/41/Flag_of_Austria.svg
8,Azerbaijan,Asia,506.000000,34107.000000,0,Autocracy,https://upload.wikimedia.org/wikipedia/commons/d/dd/Flag_of_Azerbaijan.svg
9,Bahrain,Asia,170.000000,46430.000000,0,Autocracy,https://upload.wikimedia.org/wikipedia/commons/2/2c/Flag_of_Bahrain.svg


In [281]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

def row_class(score):
    return '<td data-score="' + str(score) + '">' + str(score) + '</td>'

In [268]:
# covid_table.to_html(escape=False, formatters=dict(ImageURL=path_to_image_html))

In [313]:
style = '''
<style>
table.dataframe {
    width: 100%;
}

table.dataframe tr {
    border-left: 1px solid rgba(255,255,255,.75);
}

table.dataframe th {
    color: white;
    background-color: #6e6e6e;
    border-left: 1px solid rgba(255,255,255,.75);
}

table.dataframe td {
    font-weight: bold;
    letter-spacing: 1px;
    padding: 12px 8px;
    border-top: 1px solid rgba(255,255,255,0.5);
    border-left: 1px solid rgba(255,255,255,0.5);
}

table.dataframe td[data-score] {
    display: none;
}

table.dataframe td[data-score="0"],
table.dataframe td[data-score="0"]~ * {
    background-color: rgb(227,6,19);
    color: #e6e6e6;
}

table.dataframe td[data-score="1"],
table.dataframe td[data-score="1"]~ * {
    background-color: rgb(227, 126, 2);
    color: #e6e6e6;
}

table.dataframe td[data-score="2"],
table.dataframe td[data-score="2"]~ * {
    background-color: rgb(58, 207, 107);
}

table.dataframe td[data-score="3"],
table.dataframe td[data-score="3"]~ * {
    background-color: rgb(39, 141, 204);
}

table.dataframe td[data-score="3"],
table.dataframe td[data-score="3"]~ * {
    background-color: rgb(39, 141, 204);
}

table.dataframe tr > *:nth-child(1),
table.dataframe tr > *:nth-child(2) {
    display: none;
}
</style>
'''

covid = HTML(style + covid_table.to_html(escape=False,formatters=dict(Flags=path_to_image_html, score= row_class)))
covid

,score,Country,Continent,Total deaths,Total cases,Regimes,Flags
0,1,Afghanistan,Asia,1370.0,37551.0,Closed Anocracy,
1,3,Albania,Europe,225.0,7260.0,Democracy,
2,2,Algeria,Africa,1360.0,38133.0,Open Anocracy,
3,1,Angola,Africa,86.0,1852.0,Closed Anocracy,
4,3,Argentina,South America,5565.0,282424.0,Democracy,
5,2,Armenia,Asia,817.0,41495.0,Open Anocracy,
6,3,Australia,Oceania,379.0,23035.0,Democracy,
7,3,Austria,Europe,728.0,23211.0,Democracy,
8,0,Azerbaijan,Asia,506.0,34107.0,Autocracy,
9,0,Bahrain,Asia,170.0,46430.0,Autocracy,


In [315]:
html = covid.data
with open('covid_table.html', 'w') as f:
    f.write(html)